### Import

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### Read every cutoffs from each method

In [2]:
datasets_1 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/gmm/cutoffs/' + "/*."+'csv'))]
files_1 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/gmm/cutoffs/'))]

datasets_2 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/kmeans/cutoffs/' + "/*."+'csv'))]
files_2 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/kmeans/cutoffs/'))]

datasets_3 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/roc/cutoffs/' + "/*."+'csv'))]
files_3 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/roc/cutoffs/'))]

datasets_4 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/tertile/cutoffs/' + "/*."+'csv'))]
files_4 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/tertile/cutoffs/'))]

datasets_5 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/mean_sd/cutoffs/' + "/*."+'csv'))]
files_5 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/mean_sd/cutoffs/'))]

In [3]:
# make a dictionary that contains all cutoffs as a dataframe
gmm = dict()
kmeans = dict()
roc = dict()
tertile = dict()
mean_sd = dict()

for file_, name_ in zip(datasets_1, files_1):
    gmm[name_] = file_
    
for file_, name_ in zip(datasets_2, files_2):
    kmeans[name_] = file_
    
for file_, name_ in zip(datasets_3, files_3):
    roc[name_] = file_
    
for file_, name_ in zip(datasets_4, files_4):
    tertile[name_] = file_
    
for file_, name_ in zip(datasets_5, files_5):
    mean_sd[name_] = file_

In [4]:
#rename columns before combining the dataframes 
for i in kmeans:
    for column in kmeans[i].columns:
        kmeans[i].rename(columns={column: column.split('_')[0]}, inplace=True)

### Preprocess the cutoffs
* make a dictionary with all the cutoffs
    * keys would be the cohort names
    * values would be a dataframe with all the cutoffs of CSF biomarkers

In [5]:
def cohort_based_cutoffs(method_dfs):
    """ """

#     biomarkers = method_dfs['gmm_cutoffs_0'].columns.to_list()
    cutpoints = {i: pd.DataFrame(index=list(range(1000)), 
                columns=method_dfs[list(method_dfs.keys())[0]].columns) 
                for i in method_dfs[list(method_dfs.keys())[0]].index}

    for bioma in cutpoints['ADNI'].columns:
        
        for cohort in method_dfs[list(method_dfs.keys())[0]].index:
            
            for i in method_dfs:
                cutpoints[cohort].loc[int(i.split('_')[-1]), bioma] = method_dfs[i].loc[cohort, bioma]
                
    return cutpoints

In [16]:
gmm_new = cohort_based_cutoffs(gmm)
kmeans_new = cohort_based_cutoffs(kmeans)
roc_new = cohort_based_cutoffs(roc)
tertile_new = cohort_based_cutoffs(tertile)
mean_sd_new = cohort_based_cutoffs(mean_sd)

### Calculate the confidence intervals of CSF cutoffs for each method 

In [17]:
# results = {i: pd.DataFrame(index=['GMM', 'ROC', 'K-Means', 'Tertile', 'Mean ±2 SD'], 
#            columns=gmm[list(gmm.keys())[0]].columns) 
#             for i in gmm[list(gmm.keys())[0]].index}

results = {i: pd.DataFrame(index=gmm[list(gmm.keys())[0]].index, 
           columns=gmm[list(gmm.keys())[0]].columns) 
            for i in ['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD']}

results_percentage = {i: pd.DataFrame(index=gmm[list(gmm.keys())[0]].index, 
           columns=gmm[list(gmm.keys())[0]].columns) 
            for i in ['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD']}

In [18]:
def mean_confidence_interval(methodologies, results, confidence=0.95):
    
    
    for method in methodologies:    
        datasets_ = methodologies[method]

        for i in datasets_:

            for col in datasets_[i].columns:
    #             data = datasets_[i][col].to_list()
                a = 1.0 * np.array(datasets_[i][col])
                n = len(a)
                m, se = np.nanmean(a, 0), stats.sem(a, nan_policy="omit")
                h = se * stats.t.ppf((1 + confidence) / 2., n-1)
#                 results[i].loc[method, col] = f'{round(m, 2)} [{round(m-h, 2)}, {round(m+h, 2)}]'
                results[method].loc[i, col] = f'{round(m, 1)} [{round(m-h, 1)}, {round(m+h, 1)}]'
                results_percentage[method].loc[i, col] = f'{round((((m+h) - (m-h))/m) * 100, 2)} %'

In [19]:
methodologies = {'GMM': gmm_new, 'K-Means': kmeans_new, 'ROC': roc_new, 'Tertile': tertile_new, 
                 'Mean ±2 SD': mean_sd_new}

In [20]:
mean_confidence_interval(methodologies, results)

In [22]:
results = pd.concat(results, axis=1)

In [24]:
results.to_csv("../results/bootstrap/confidence_intervals.csv")

In [25]:
results

GMM                        \
                 A-beta 1-42 in CSF           pTau in CSF   
ADNI           986.9 [986.0, 987.9]     34.7 [34.6, 34.8]   
EPAD        1035.7 [1035.0, 1036.4]     27.8 [27.7, 27.8]   
AIBL           749.0 [743.2, 754.8]     87.8 [86.8, 88.9]   
ARWIBO         589.2 [585.0, 593.5]  155.0 [148.2, 161.9]   
EDSD           782.2 [776.1, 788.3]  119.2 [117.7, 120.8]   
PREVENT-AD  1187.4 [1158.6, 1216.2]     71.1 [70.3, 72.0]   
PharmaCog      792.9 [791.1, 794.7]     93.0 [92.7, 93.3]   
NACC_ELISA     594.1 [592.6, 595.7]     81.7 [80.9, 82.5]   
EMIF_ELISA     742.1 [741.2, 743.0]     82.1 [81.9, 82.3]   
NACC_XMAP      298.7 [297.0, 300.4]     60.8 [59.9, 61.7]   
EMIF_XMAP      386.9 [384.2, 389.5]     51.6 [51.2, 52.1]   
DOD-ADNI    1412.8 [1394.9, 1430.8]     26.6 [26.4, 26.7]   
JADNI          414.7 [412.5, 416.9]     61.1 [60.8, 61.5]   

                                                  K-Means                     \
                     tTau in CSF       A-beta 1-42 in CSF        pTau in CSF   
ADNI        356.9 [356.1, 357.7]     972.6 [971.6, 973.6]  28.1 [28.0, 28.1]   
EPAD        307.0 [306.6, 307.4]  1033.3 [1032.6, 1034.0]  19.8 [19.8, 19.9]   
AIBL        604.5 [597.0, 612.0]     684.0 [681.2, 686.7]  77.7 [77.0, 78.4]   
ARWIBO      531.6 [526.7, 536.4]     523.8 [521.7, 526.0]  69.1 [68.9, 69.4]   
EDSD        584.8 [576.2, 593.5]     741.8 [736.1, 747.5]  86.7 [86.0, 87.4]   
PREVENT-AD  466.5 [457.2, 475.8]  1146.4 [1144.1, 1148.8]  50.8 [50.6, 51.0]   
PharmaCog   773.4 [750.8, 796.0]     762.2 [759.5, 764.8]  68.6 [68.0, 69.2]   
NACC_ELISA  564.6 [552.4, 576.9]     586.1 [584.9, 587.2]  61.6 [61.4, 61.8]   
EMIF_ELISA  526.6 [524.2, 528.9]     697.7 [695.2, 700.1]  64.9 [64.8, 65.0]   
NACC_XMAP      94.5 [93.8, 95.3]     290.7 [288.4, 293.0]  40.1 [40.0, 40.3]   
EMIF_XMAP   227.7 [220.8, 234.5]     363.3 [362.3, 364.2]  37.5 [37.4, 37.6]   
DOD-ADNI    304.0 [302.3, 305.7]  1273.5 [1267.1, 1279.8]  22.4 [22.3, 22.5]   
JADNI       133.9 [132.6, 135.2]     392.5 [390.6, 394.4]  59.9 [59.6, 60.1]   

                                                  Tertile                     \
                     tTau in CSF       A-beta 1-42 in CSF        pTau in CSF   
ADNI        288.2 [287.8, 288.6]     833.3 [831.2, 835.3]  21.8 [21.8, 21.9]   
EPAD        223.7 [223.5, 223.9]     884.5 [883.5, 885.4]  19.9 [19.8, 19.9]   
AIBL        530.6 [523.3, 537.9]                      NaN                NaN   
ARWIBO      432.1 [428.8, 435.4]                      NaN                NaN   
EDSD        547.6 [543.0, 552.2]                      NaN                NaN   
PREVENT-AD  308.8 [306.6, 310.9]  1098.7 [1096.8, 1100.6]  53.0 [52.9, 53.1]   
PharmaCog   466.9 [464.6, 469.2]                      NaN                NaN   
NACC_ELISA  492.1 [490.2, 493.9]     625.5 [623.6, 627.5]  48.3 [48.1, 48.4]   
EMIF_ELISA  432.5 [431.7, 433.4]     541.0 [539.7, 542.2]  52.3 [52.2, 52.4]   
NACC_XMAP      70.2 [70.0, 70.4]     248.5 [247.7, 249.3]  36.5 [36.4, 36.6]   
EMIF_XMAP   128.9 [128.3, 129.5]     446.7 [444.9, 448.4]  31.4 [31.1, 31.6]   
DOD-ADNI    249.8 [248.8, 250.8]     928.4 [924.4, 932.4]  19.8 [19.8, 19.9]   
JADNI       129.0 [127.9, 130.0]     392.7 [390.9, 394.5]  38.5 [38.5, 38.6]   

                                                   ROC                     \
                     tTau in CSF    A-beta 1-42 in CSF        pTau in CSF   
ADNI        240.1 [239.5, 240.6]  819.4 [815.8, 823.1]  23.6 [23.5, 23.7]   
EPAD        228.2 [227.9, 228.5]  684.7 [680.5, 688.9]  23.9 [23.6, 24.3]   
AIBL                         NaN                   NaN                NaN   
ARWIBO                       NaN                   NaN                NaN   
EDSD                         NaN                   NaN                NaN   
PREVENT-AD  303.3 [302.5, 304.1]                   NaN                NaN   
PharmaCog                    NaN                   NaN                NaN   
NACC_ELISA  371.3 [36

In [26]:
pd.concat(results_percentage, axis=1)

GMM                                    K-Means  \
           A-beta 1-42 in CSF pTau in CSF tTau in CSF A-beta 1-42 in CSF   
ADNI                   0.19 %      0.64 %      0.47 %             0.21 %   
EPAD                   0.14 %      0.33 %      0.25 %             0.13 %   
AIBL                   1.54 %      2.39 %      2.48 %              0.8 %   
ARWIBO                 1.44 %      8.85 %      1.84 %             0.81 %   
EDSD                   1.56 %      2.59 %      2.95 %             1.54 %   
PREVENT-AD             4.84 %      2.46 %      3.99 %             0.41 %   
PharmaCog              0.45 %      0.61 %      5.85 %             0.69 %   
NACC_ELISA             0.51 %      1.99 %      4.33 %             0.39 %   
EMIF_ELISA             0.24 %      0.46 %       0.9 %              0.7 %   
NACC_XMAP              1.15 %      2.99 %      1.54 %             1.56 %   
EMIF_XMAP              1.36 %      1.77 %      6.02 %             0.52 %   
DOD-ADNI               2.55 %       0.9 %      1.13 %             0.99 %   
JADNI                  1.06 %      1.05 %      1.93 %             0.96 %   

                                              Tertile                          \
           pTau in CSF tTau in CSF A-beta 1-42 in CSF pTau in CSF tTau in CSF   
ADNI            0.25 %      0.26 %             0.48 %       0.5 %      0.48 %   
EPAD            0.22 %       0.2 %             0.21 %      0.28 %      0.27 %   
AIBL            1.81 %      2.75 %                NaN         NaN         NaN   
ARWIBO          0.72 %      1.52 %                NaN         NaN         NaN   
EDSD            1.61 %      1.68 %                NaN         NaN         NaN   
PREVENT-AD       0.9 %      1.39 %             0.35 %      0.52 %      0.51 %   
PharmaCog       1.71 %      0.98 %                NaN         NaN         NaN   
NACC_ELISA      0.69 %      0.76 %             0.62 %      0.55 %       0.9 %   
EMIF_ELISA      0.27 %      0.38 %             0.45 %      0.39 %      0.57 %   
NACC_XMAP       0.64 %      0.64 %             0.63 %      0.38 %      0.61 %   
EMIF_XMAP       0.53 %      0.87 %             0.77 %      1.54 %      1.37 %   
DOD-ADNI        0.97 %      0.81 %             0.85 %      0.65 %      0.62 %   
JADNI           0.82 %      1.66 %             0.92 %      0.45 %       0.7 %   

                          ROC                                 Mean ±2 SD  \
           A-beta 1-42 in CSF pTau in CSF tTau in CSF A-beta 1-42 in CSF   
ADNI                   0.89 %      0.85 %      0.77 %              1.4 %   
EPAD                   1.23 %      3.02 %      3.39 %             0.54 %   
AIBL                      NaN         NaN         NaN                NaN   
ARWIBO                    NaN         NaN         NaN                NaN   
EDSD                      NaN         NaN         NaN                NaN   
PREVENT-AD                NaN         NaN         NaN             1.33 %   
PharmaCog                 NaN         NaN         NaN                NaN   
NACC_ELISA             1.47 %      1.16 %      1.82 %             1.22 %   
EMIF_ELISA             1.51 %      1.51 %      1.22 %             1.78 %   
NACC_XMAP              0.48 %      0.92 %       1.8 %             8.67 %   
EMIF_XMAP              1.14 %      1.71 %      1.14 %             2.45 %   
DOD-ADNI                  NaN         NaN         NaN             2.66 %   
JADNI                  0.85 %      1.09 %      0.41 %             1.84 %   

                                    
           pTau in CSF tTau in CSF  
ADNI            0.53 %      0.45 %  
EPAD            0.43 %      0.34 %  
AIBL               NaN         NaN  
ARWIBO             NaN         NaN  
EDSD               NaN         NaN  
PREVENT-AD      0.87 %      1.29 %  
PharmaCog          NaN         NaN  
NACC_ELISA      1.11 %      0.93 %  
EMIF_ELISA      1.08 %      2.56 %  
NACC_XMAP       1.19 %      0.97 %  
EMIF_XMAP       1.38 %      0.84 %  
DOD-ADNI        0.84 %      0.67 %  
JADNI     

In [53]:
results_percentage['GMM']

,A-beta 1-42 in CSF,pTau in CSF,tTau in CSF
ADNI,0.19,0.64,0.47
EPAD,0.14,0.33,0.25
AIBL,1.54,2.39,2.48
ARWIBO,1.44,8.85,1.84
EDSD,1.56,2.59,2.95
PREVENT-AD,4.84,2.46,3.99
PharmaCog,0.45,0.61,5.85
NACC_ELISA,0.51,1.99,4.33
EMIF_ELISA,0.24,0.46,0.9
NACC_XMAP,1.15,2.99,1.54
